In [2]:
import sys, os

os.chdir("../")
sys.path.append(os.getcwd())


In [ ]:
import os
import shutil
import time
import subprocess
import threading
import wandb
import logging
import asyncio
import base64
import json
import hashlib
import hmac
from io import BytesIO
from typing import List, Dict, Optional
from datetime import datetime
import zipfile
from pathlib import Path

# import torch
from huggingface_hub import HfFolder, login
from centrifuge import CentrifugeError, Client, ClientEventHandler, SubscriptionEventHandler
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
from diffusers import DiffusionPipeline
from aixblock_ml.model import AIxBlockMLBase

from logging_class import start_queue, write_log
from function_ml import connect_project, download_dataset, upload_checkpoint
from misc import get_device_count
import constants as const

In [ ]:
HfFolder.save_token(const.HF_TOKEN)
login(token=const.HF_ACCESS_TOKEN)
# wandb.login('allow',const.WANDB_TOKEN)
# print("Login successful")

In [94]:
dataset_path="/mnt/d/workplace/workplace_aixblock/ml-backend/flux/data/extract"
host_name="https://dev-us-west-1.aixblock.io"
token="ebcf0ceda01518700f41dfa234b6f4aaea0b57af"
project_id=432
clone_dir="/mnt/d/workplace/workplace_aixblock/ml-backend/flux/data"
# dataset_id="coco_manual"
# dataset_id="yolo_manual"
# dataset_id="json_s3_export"
dataset_id="json_default"


In [95]:
project = connect_project(host_name, token, project_id)

In [96]:
def is_platform_json_file(filename, folder_path):
    # TODO: use different approach other than filename
    files = [f for f in folder_path.iterdir() if f.is_file()]
    print(filename == 'result.json', len(files) == 1)
    if filename == 'result.json' and len(files) == 1:
        return True
    else:
        return False

def clean_folder(folder_path):
    folder = Path(folder_path)
    if not folder.exists():
        print("Folder does not exist.")
        return

    for item in folder.iterdir():
        try:
            if item.is_file() or item.is_symlink():
                item.unlink()  # Delete file or symbolic link
            elif item.is_dir():
                shutil.rmtree(item)  # Delete folder and its contents
        except Exception as e:
            print(f"Failed to delete {item}: {e}")


def get_first_json_file(folder_path):
    folder = Path(folder_path)
    for file in folder.iterdir():
        if file.is_file() and file.suffix == '.json':
            return file.name, file  
    return None , None


In [97]:
data_path = os.path.join(clone_dir, "data_zip")
os.makedirs(data_path, exist_ok=True)
# dataset_name = download_dataset(project, dataset_id, data_path)
dataset_name = dataset_id + '.zip'
if dataset_name:
    data_zip_dir = os.path.join(data_path, dataset_name)
    with zipfile.ZipFile(data_zip_dir, 'r') as zip_ref:
        clean_folder(dataset_path)
        zip_ref.extractall(path=dataset_path)
        # conversion pipeline for aixblock storage, json file
        # convert json file into specified format



In [98]:
json_file, json_file_dir = get_first_json_file(dataset_path)
print(json_file,json_file_dir)
if json_file and is_platform_json_file(json_file, json_file_dir.parent):
    with open(json_file_dir) as f:
        jsonl_1 = json.load(f)
        jsonl_2 = [{"image": data["data"].get("images"), "prompt": data.get("prompt")} for data in jsonl_1]
        with open(json_file_dir, 'w') as f:
            json.dump(jsonl_2, f)
        print("modified json to usable format")
        

json_default.json /mnt/d/workplace/workplace_aixblock/ml-backend/flux/data/extract/json_default.json
False True


In [ ]:
!accelerate launch ./train_dreambooth_flux.py \
  --pretrained_model_name_or_path="hf-internal-testing/tiny-flux-pipe"  \
  --dataset_name="lambdalabs/naruto-blip-captions" #"crystantine/fluxgym" \
  --output_dir="./models/1" \
  --instance_prompt="dog" \
  --resolution=128 \
  --train_batch_size=100 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1
  --push_to_hub

In [ ]:
# COCO
!accelerate launch train_dreambooth_flux.py \
  --pretrained_model_name_or_path="hf-internal-testing/tiny-flux-pipe" \
  --dataset_name="./data/extract" \
  --output_dir="./models/1" \
  --instance_prompt="dog" \
  --resolution=128 \
  --train_batch_size=8 \
  --learning_rate=5e-6 \
  --max_train_steps=1 \
  --push_to_hub

In [ ]:
# YOLO
!accelerate launch train_dreambooth_flux.py \
  --pretrained_model_name_or_path="hf-internal-testing/tiny-flux-pipe" \
  --dataset_name="./data/extract" \
  --output_dir="./models/2" \
  --instance_prompt="dog" \
  --resolution=128 \
  --train_batch_size=8 \
  --learning_rate=5e-6 \
  --max_train_steps=1\
  --push_to_hub


In [ ]:
# json export file from s3
!accelerate launch train_dreambooth_flux.py \
  --pretrained_model_name_or_path="hf-internal-testing/tiny-flux-pipe" \
  --dataset_name="./data/extract" \
  --output_dir="./models/3" \
  --instance_prompt="dog" \
  --resolution=128 \
  --train_batch_size=8 \
  --learning_rate=5e-6 \
  --max_train_steps=1\
  --push_to_hub

In [ ]:
# json standard format, custom column
!accelerate launch train_dreambooth_flux.py \
  --pretrained_model_name_or_path="hf-internal-testing/tiny-flux-pipe" \
  --dataset_name="./data/extract" \
  --output_dir="./models/4" \
  --instance_prompt="dog" \
  --image_column="my-image-url" \
  --caption_column="my-prompt" \
  --resolution=128 \
  --train_batch_size=8 \
  --learning_rate=5e-6 \
  --num_train_epochs=5\
  --max_train_steps=100
  # --push_to_hub

2025-02-16 12:48:15.843139: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-16 12:48:15.967983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739710096.017916  183706 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739710096.034040  183706 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 12:48:16.174959: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
from datasets import load_dataset, Image

data_files = {
    "train": "./validation/images",
    "validation": "./validation/images",
    "test": "./validation/images"
}

dataset = load_dataset(
  "data/extract",
  # "data/extract",
    # "Rapidata/Flux_SD3_MJ_Dalle_Human_Coherence_Dataset",
    # "crystantine/fluxgym",
    # "data/1",
    # "k-mktr/improved-flux-prompts",
)
# dataset = dataset.cast_column("image", Image())

# dataset = load_dataset(
#   "imagefolder",
#   data_dir= "data/extract",
#     # "Rapidata/Flux_SD3_MJ_Dalle_Human_Coherence_Dataset",
#     # "crystantine/fluxgym",
#     # "data/1",
#     # "k-mktr/improved-flux-prompts",
# )
# dataset = dataset.cast_column("image", Image())

In [ ]:
print(1)

In [ ]:
# dataset = load_dataset("nyu-mll/glue", "mrpc", split="train")
dataset